In [43]:
import cv2
import numpy as np
from PIL import Image

In [44]:
capture = cv2.VideoCapture(0)

In [45]:
def get_limits(color):
    # Convert color to uint8 array
    color_array = np.uint8([[color]])
    
    # Convert color array to HSV color space
    hsv_color = cv2.cvtColor(color_array, cv2.COLOR_BGR2HSV)
    
    # Extract hue value
    hue = hsv_color[0][0][0]
    
    # Handle red hue wrap-around
    if hue >= 165:
        # Set lower and upper limits for hue range
        lower_limit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upper_limit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:
        # Set lower and upper limits for hue range
        lower_limit = np.array([0, 100, 100], dtype=np.uint8)
        upper_limit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        # Set lower and upper limits for hue range
        lower_limit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upper_limit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    
    # Return the lower and upper limits
    return lower_limit, upper_limit

In [46]:
color = [0, 255, 255]

while True:
    # Read frame from video capture
    ret, frame = capture.read()
    
    # Convert frame to HSV color space
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get lower and upper limits for the specified color
    lower_limit, upper_limit = get_limits(color)
    
    # Create a mask based on the color range
    the_mask = cv2.inRange(hsv_image, lower_limit, upper_limit)
    
    # Convert the mask to an Image object
    mask_image = Image.fromarray(the_mask)
    
    # Get the bounding box of the mask
    bbox = mask_image.getbbox()
    
    if bbox is not None:
        # Extract the coordinates of the bounding box
        x1, y1, x2, y2 = bbox
        
        # Draw a rectangle on the frame around the detected object
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
    
    # Display the frame
    cv2.imshow('frame', frame)
    
    # Check if 'q' key is pressed to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all windows
capture.release()
cv2.destroyAllWindows()